In [1]:
import os

In [2]:
%pwd

'c:\\Users\\RedHat\\Desktop\\Projects\\Disease_Classification\\Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\RedHat\\Desktop\\Projects\\Disease_Classification\\Disease-Classification-Project'

In [5]:
## creating a custom return type using entity 
## this will be moved into the entity folder later

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.reusables import read_yaml, create_directories

In [7]:
## This will be moved into the configuration.py folder later

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir, 
            source_url=config.source_url, 
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
        )
            
        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.reusables import get_size


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following headers: \n{headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists with size: \
                        {get_size(Path(self.config.local_data_file))}")
            
    
    def unzip_file(self):
        """
        Unzips the downloaded file into the data directory

        zip_file_path: str
        Function returns None

        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

[2024-03-08 14:45:19,304: INFO: reusables: yaml file: config\config.yaml loaded successfully]
[2024-03-08 14:45:19,309: INFO: reusables: yaml file: params.yaml loaded successfully]
[2024-03-08 14:45:19,312: INFO: reusables: created directory at: artifacts]
[2024-03-08 14:45:19,315: INFO: reusables: created directory at: artifacts/data_ingestion]
[2024-03-08 14:45:28,439: INFO: 4205889240: artifacts/data_ingestion/data.zip downloaded with the following headers: 
Connection: close
Content-Length: 24438364
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a1b5a8b43c38ab2e5dcb3abc03ba6dc1e5734444af8a34aefc5344034671d091"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1BD8:3A5403:14A21AE:159FA35:65EB2481
Accept-Ranges: bytes
Date: Fri, 08 Mar 2024 14:45:23 GMT
Via: 1.1 varnish
X-Served-By: cache-